In [14]:
import pandas as pd
from libs.recommend_user_item import train_svd_model, convert_ratings_to_trainset
%load_ext autoreload
%autoreload 2

params = {
    "n_epochs": 50,
    "lr_all": 0.013795417732518363,
    "reg_all": 0.8966771292853577,
    "n_factors": 17}
n_item = 10
n_user = 10
top_n = 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('Data/download/ratings.csv').drop(columns='ts')
df

,user_uid,element_uid,rating
0,571252,1364,10
1,63140,3037,10
2,443817,4363,8
3,359870,1364,10
4,359870,3578,9
...,...,...,...
438785,170707,1539,8
438786,32659,6127,8
438787,353752,3336,8
438788,492350,7984,7


In [8]:
(df['element_uid'].value_counts() > n_item).sum()

3396

In [7]:
(df['user_uid'].value_counts() > n_user).sum()

8630

In [9]:
user_counts = df['user_uid'].value_counts()
movie_counts = df['element_uid'].value_counts()

# Фильтруем пользователей, которые оценили меньше 2 фильмов
users_to_keep = user_counts[user_counts > n_user].index
df_filtered_users = df[df['user_uid'].isin(users_to_keep)]

# Фильтруем фильмы, которые оценили меньше 10 раз
movies_to_keep = movie_counts[movie_counts > n_item].index
df_filtered = df_filtered_users[df_filtered_users['element_uid'].isin(movies_to_keep)]
df_filtered

,user_uid,element_uid,rating
1,63140,3037,10
2,443817,4363,8
6,230987,8273,10
9,36122,7587,6
12,281031,8739,7
...,...,...,...
438779,287021,3336,10
438782,288896,2362,8
438783,104636,7132,8
438785,170707,1539,8


In [41]:
uniq_100 = df_filtered['user_uid'].unique()[:100]
df_filtered = df_filtered[df_filtered['user_uid'].isin(uniq_100)]
df_filtered

,user_uid,element_uid,rating
0,571252,1364,10
1,63140,3037,10
2,443817,4363,8
3,359870,1364,10
4,359870,3578,9
...,...,...,...
435940,20211,2075,8
436755,20211,9702,8
437670,61690,5266,8
437812,433795,8919,9


In [26]:
df.groupby('element_uid', as_index=False)['rating'].sum().sort_values(by='rating', ascending=False).reset_index(drop=True)['element_uid'][:top_n].to_list()

[747, 2714, 5616, 3336, 2245]

Проверяем, есть ли среди пользователей те, кто оценил один фильм дважды. Таких пользователей нет, можно работать с данными без группировки и учёта времени.

In [10]:
data = convert_ratings_to_trainset(df_filtered)
data

In [11]:
rec = train_svd_model(data_prep_df=data,
                      best_params=params,
                      n_pred=5,
                )
rec

100%|██████████| 8630/8630 [00:09<00:00, 895.32it/s] 


,item_id,rating,user_id
0,6461,9.537187,63140
1,3223,9.477430,63140
2,4660,9.471472,63140
3,8338,9.436611,63140
4,4154,9.380242,63140
...,...,...,...
43145,6461,9.956584,176015
43146,3223,9.896807,176015
43147,4660,9.890852,176015
43148,8338,9.855988,176015


In [12]:
rec.to_pickle('okko_rec.pickle')

In [25]:
import pickle

with open('cache/okko_rec.pickle', 'rb') as f:
    cache = pickle.load(f)
cache

,item_id,rating,user_id
0,6461,9.537187,63140
1,3223,9.477430,63140
2,4660,9.471472,63140
3,8338,9.436611,63140
4,4154,9.380242,63140
...,...,...,...
43145,6461,9.956584,176015
43146,3223,9.896807,176015
43147,4660,9.890852,176015
43148,8338,9.855988,176015


In [22]:
from libs.http_actions.model_cache import get_top_similar_by_user

get_top_similar_by_user(63140)

cache_file D:\Документы\Рабочее\ML\Другое\Okko_rec_sys\cache\okko_rec.pickle
user2item_df        item_id    rating  user_id
0         6461  9.537187    63140
1         3223  9.477430    63140
2         4660  9.471472    63140
3         8338  9.436611    63140
4         4154  9.380242    63140
...        ...       ...      ...
43145     6461  9.956584   176015
43146     3223  9.896807   176015
43147     4660  9.890852   176015
43148     8338  9.855988   176015
43149     4154  9.799623   176015

[43150 rows x 3 columns]


array([6461, 3223, 4660, 8338, 4154], dtype=int64)